# <center>[🦏 Автоматический стэкинг и блендинг](https://stepik.org/lesson/872530/)</center>

## Импортируем библиотеки

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import lightgbm as lgbm
import xgboost as xgb
import catboost as cb

In [3]:
from data_process import (RANDOM_SEED, PREDICTIONS_DIR, get_max_num, DataTransform,
                          make_predict, add_info_to_log, merge_submits, LOCAL_FILE)

from print_time import print_time, print_msg
from set_all_seeds import set_all_seeds


set_all_seeds(seed=RANDOM_SEED)

## Считываем данные

In [4]:
sub_pref = 'st_'

numeric_columns = []

cat_columns = ['Pclass',
               # 'SibSp',  # Количество Братьев (Сестер) / Супругов на борту
               # 'Parch',  # Количество Родителей / Детей на борту
               'Embarked',
               'title',
               # 'family',  # Количество всех членов семьи
               'deck',  # Палуба символ
               'embarked',  # индекс Порт посадки
               'title_cat',
               'idx_age',  # разбивка возрастов по категориям
               'idx_family',  # разбивка семей по категориям
               # 'len_ticket',  # Длина номера билета
               'fst_sym_ticket',
               # 'ticket_freq',  # частота использования билета
               'idx_deck',  # Индекс Палубы
               'idx_fare',  # Индекс Тарифа
               ]

# Чтение и предобработка данных
data_cls = DataTransform(use_catboost=True,
                         category_columns=cat_columns,
                         drop_first=False,
                         # numeric_columns=numeric_columns, scaler=StandardScaler,
                         )

train_df, test_df = data_cls.make_agg_data()

train_df.set_index('PassengerId', inplace=True)
test_df.set_index('PassengerId', inplace=True)

# # Добавление группировок по целевому признаку
# train_df = data_cls.fit_transform(train_df)
# test_df = data_cls.transform(test_df)

features2drop = ['Survived']

exclude_columns = [
    'idx_age',  # разбивка возрастов по категориям
    'idx_fare',  # Индекс Тарифа
    'idx_ticket',
    'idx_len_ticket',
    # 'len_ticket',  # Длина номера билета
]

exclude_columns.extend(data_cls.exclude_columns)

model_columns = test_df.columns.to_list()

model_columns = [col for col in model_columns if col not in exclude_columns]

# Добавим в категориальные признаки те, что были посчитаны как мода
cat_columns.extend([col for col in model_columns if col.upper().startswith('MODE_')])

model_columns = [col for col in model_columns if col not in exclude_columns]
cat_columns = [col for col in cat_columns if col in model_columns + features2drop]
num_columns = [col for col in model_columns if col not in cat_columns + features2drop]

exclude_columns = features2drop + exclude_columns

print('Обучаюсь на колонках:', model_columns)
print('Категорийные колонки:', cat_columns)
print('Исключенные колонки:', exclude_columns)

print(f'Размер train_df = {train_df.shape}, test = {test_df.shape}')

train = train_df[model_columns].drop(columns=features2drop, errors='ignore')
target = train_df['Survived']
test_df = test_df[model_columns].copy()

for col in cat_columns:
    train[col] = train[col].astype('category')
    test_df[col] = test_df[col].astype('category')

print('train.shape', train.shape, 'пропусков:', train.isna().sum().sum())
print('test.shape', test_df.drop(columns=features2drop, errors='ignore').shape,
      'пропусков:', test_df.isna().sum().sum())

Читаю подготовленные данные...
Время обработки: 0.0 сек
Обучаюсь на колонках: ['Pclass', 'SibSp', 'Parch', 'title', 'len_name', 'alone', 'family', 'delta', 'fare', 'age', 'sex', 'embarked', 'cabin', 'title_cat', 'retirer', 'idx_family', 'len_ticket', 'fst_sym_ticket', 'ticket_freq', 'ticket_fare', 'idx_deck', 'fare_zero', 'fare_log', 'double_name']
Категорийные колонки: ['Pclass', 'title', 'embarked', 'title_cat', 'idx_family', 'fst_sym_ticket', 'idx_deck']
Исключенные колонки: ['Survived', 'idx_age', 'idx_fare', 'idx_ticket', 'idx_len_ticket', 'learn', 'Name', 'Sex', 'Age', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'new_deck', 'is_cabin', 'name_delta', 'deck', 'ticket']
Размер train_df = (891, 41), test = (418, 40)
train.shape (891, 24) пропусков: 0
test.shape (418, 24) пропусков: 0


### Разделим выборку на валидационную и обучающую

In [5]:
test_size = 0.2

num_folds = 5

test_size = round(test_size, 2)

print(f'valid_size: {test_size}, SEED={RANDOM_SEED}')

# Создаем комбинированный столбец для стратификации
train_df['survd_sex'] = train_df['Survived'].astype(str) + '_' + train['sex'].astype(str)

stratified = None
stratified = ['Survived']
# stratified = ['survd_sex']

stratified_target = train_df[stratified] if stratified else None

# Разделение на обучающую и валидационную выборки

X_train, X_valid, y_train, y_valid = train_test_split(train, target,
                                                      test_size=test_size,
                                                      stratify=stratified_target,
                                                      random_state=RANDOM_SEED)

splited = X_train, X_valid, y_train, y_valid

print('X_train.shape', X_train.shape)
print('X_valid.shape', X_valid.shape)

valid_size: 0.2, SEED=127
X_train.shape (712, 24)
X_valid.shape (179, 24)


# <center> ☘️ Объявим 3 модели

#  😺🚀 Модель `CatBoost`

In [6]:
params_cat = {'iterations': 1000,
              'learning_rate': 0.12562363132860913,
              'depth': 5,
              'l2_leaf_reg': 22.283641064716228,
              'rsm': 0.6522619525341526,
              'loss_function': 'Logloss',
              'border_count': 132,
              'leaf_estimation_method': 'Gradient',
              'random_seed': RANDOM_SEED,
              'one_hot_max_size': 8,
              'random_strength': 4.170112970772482,
              'eval_metric': 'Accuracy',
              'bagging_temperature': 7.796458100670984,
              'boosting_type': 'Ordered',
              'bootstrap_type': 'Bayesian',
              'early_stopping_rounds': 100,
              'cat_features': cat_columns,
              }

In [7]:
cat_model = cb.CatBoostClassifier(**params_cat)

# 🦄🎳 Модель `LightGBM`

In [8]:
# Указание категориальных признаков (если есть)
categorical_features = [train.columns.get_loc(col) for col in cat_columns] if cat_columns else None

params_lgbm = {'boosting_type': 'dart',
               'colsample_bytree': 0.6459964090228278,
               'importance_type': 'split',
               'learning_rate': 0.2816036393016932,
               'max_depth': 4,
               'min_child_samples': 100,
               'min_child_weight': 0.001,
               'min_split_gain': 0.0,
               'n_estimators': 100,
               'num_leaves': 85,
               'objective': 'binary',
               'random_state': RANDOM_SEED,
               'random_seed': RANDOM_SEED,
               'reg_alpha': 1.5946003373810458,
               'reg_lambda': 7.005261713251808,
               'subsample': 0.8095790668625238,
               'subsample_for_bin': 200000,
               'subsample_freq': 2,
               'eval_metric': 'Accuracy',
               'verbose': -1,
               'cat_feature': categorical_features,
               }

In [9]:
lgbm_model = lgbm.LGBMClassifier(**params_lgbm)

# 👽🔱 Модель `XGBoost`

In [10]:
params_xgb = {'objective': 
              'binary:logistic',
              'booster': 'dart',
              'colsample_bytree': 0.8679507104120996,
              'enable_categorical': True,
              'eval_metric': 'logloss',
              'max_depth': 4,
              'min_child_weight': 2,
              'subsample': 0.8004843970146179,
              'random_seed': RANDOM_SEED,
              'verbose': -1,
              'eta': 0.225713349140033,
              'alpha': 2.206812214230348,
              'lambda': 4.27264717515999,
              'rate_drop': 0.052499457650865014,
              'skip_drop': 0.19868392699431042,
              }

In [11]:
xgb_model = xgb.XGBClassifier(**params_xgb)

# <center> 🥤 Построим пайплан

In [12]:
# Вспомогательные блоки организации для пайплайна
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

In [13]:
# Вспомогательные элементы для наполнения пайплайна
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

In [14]:
# Некоторые модели для построение ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

In [15]:
# Добавим визуализации
import sklearn
sklearn.set_config(display='diagram')

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

### Предобработаем данные
Под каждый тип данных заводим свой трансформер

In [16]:
categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

numerical_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

# соединим два предыдущих трансформера в один
preprocessor = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, num_columns),
    ("categorical", categorical_transformer, cat_columns)])

In [17]:
preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['SibSp', 'Parch', 'len_name', 'alone',
                                  'family', 'delta', 'fare', 'age', 'sex',
                                  'cabin', 'retirer', 'len_ticket',
                                  'ticket_freq', 'ticket_fare', 'fare_zero',
                                  'fare_log', 'double_name']),
                                ('categorical',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Pclass', 'title', 'embarked', 'title_cat',
                                  'idx_family', 'fst_sym_ticket',
                                  'idx_deck'])])

In [18]:
preprocessor.transformers[0]

('numerical',
 Pipeline(steps=[('scaler', StandardScaler())]),
 ['SibSp',
  'Parch',
  'len_name',
  'alone',
  'family',
  'delta',
  'fare',
  'age',
  'sex',
  'cabin',
  'retirer',
  'len_ticket',
  'ticket_freq',
  'ticket_fare',
  'fare_zero',
  'fare_log',
  'double_name'])

# <center> 🎓🐊 Обучим ансамбль

In [19]:
params_et = {'bootstrap': False,
             'ccp_alpha': 0.0,
             'criterion': 'gini',
             'max_depth': 7,
             'max_features': 'sqrt',
             'min_impurity_decrease': 0.0,
             'min_samples_leaf': 7,
             'min_samples_split': 6,
             'min_weight_fraction_leaf': 0.0,
             'n_estimators': 548,
             'n_jobs': -1,
             'oob_score': False,
             'random_state': RANDOM_SEED,
             'verbose': False,
             'warm_start': False}


params_rf = {'bootstrap': True,
             'ccp_alpha': 0.0,
             'criterion': 'gini',
             'max_depth': 3,
             'max_features': 'log2',
             'min_impurity_decrease': 0.0,
             'min_samples_leaf': 10,
             'min_samples_split': 4,
             'min_weight_fraction_leaf': 0.0,
             'n_estimators': 466,
             'n_jobs': -1,
             'oob_score': False,
             'random_state': RANDOM_SEED,
             'verbose': False,
             'warm_start': False,
             }


params_nn = {'batch_size': 128,
             'best_epoch': 8,
             'max_epochs': 8,
             'dropout1': 0.5544539787798769,
             'dropout2': 0.37587705852465625,
             'hidden_size': 72,
             'lr': 0.05620056948941698,
             'random_state': RANDOM_SEED,
             'show_progress': False
             }

In [20]:
from nn_torch import TabularClassifier

In [21]:
# список базовых моделей
estimators = [

    ("ExtraTrees",  make_pipeline(preprocessor, ExtraTreesClassifier(**params_et))),
    
    ("XGBoost", xgb_model),
    
    ("LightGBM", lgbm_model),
    
    ("CatBoost", cat_model),
    
    ("Random_forest",  make_pipeline(preprocessor, RandomForestClassifier(**params_rf))),
    
    ("Tabular_network",  make_pipeline(preprocessor, TabularClassifier(**params_nn))),    

]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False),
    # final_estimator=RandomForestClassifier(n_estimators = 10_000,
                                           # max_depth = 5,
                                           # verbose=False),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('ExtraTrees',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('numerical',
                                                                                  Pipeline(steps=[('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['SibSp',
                                                                                   'Parch',
                                                                                   'len_name',
                                                                                   'alone',
                                                                                   'family',
                                                                                   'delta',
                                                                                   'fare',
                                                                                   'age',
                                                                                   'sex',
                                                                                   'cabin',
                                                                                   'retirer',
                                                                                   'len_ticket',
                                                                                   'ticket_freq',
                                                                                   'ticket_fare',
                                                                                   'fare_zero',
                                                                                   'fare_log',
                                                                                   'double_name']),
                                                                                 ('categ...
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['Pclass',
                                                                                   'title',
                                                                                   'embarked',
                                                                                   'title_cat',
                                                                                   'idx_family',
                                                                                   'fst_sym_ticket',
                                                                                   'idx_deck'])])),
                                                ('tabularclassifier',
                                                 TabularClassifier(device='cpu',
                                                                   dropout1=0.5544539787798769,
                                                                   dropout2=0.37587705852465625,
                                                                   hidden_size=72,
                                                                   lr=0.05620056948941698,
                                                                   max_epochs=8,
                                                                   random_state=127))]))],
                   final_estimator=LogisticRegression(verbose=False), n_jobs=-1,
                   verbose=False)

In [22]:
stacking_classifier.fit(X_train, y_train)

StackingClassifier(estimators=[('ExtraTrees',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('numerical',
                                                                                  Pipeline(steps=[('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['SibSp',
                                                                                   'Parch',
                                                                                   'len_name',
                                                                                   'alone',
                                                                                   'family',
                                                                                   'delta',
                                                                                   'fare',
                                                                                   'age',
                                                                                   'sex',
                                                                                   'cabin',
                                                                                   'retirer',
                                                                                   'len_ticket',
                                                                                   'ticket_freq',
                                                                                   'ticket_fare',
                                                                                   'fare_zero',
                                                                                   'fare_log',
                                                                                   'double_name']),
                                                                                 ('categ...
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['Pclass',
                                                                                   'title',
                                                                                   'embarked',
                                                                                   'title_cat',
                                                                                   'idx_family',
                                                                                   'fst_sym_ticket',
                                                                                   'idx_deck'])])),
                                                ('tabularclassifier',
                                                 TabularClassifier(device='cpu',
                                                                   dropout1=0.5544539787798769,
                                                                   dropout2=0.37587705852465625,
                                                                   hidden_size=72,
                                                                   lr=0.05620056948941698,
                                                                   max_epochs=8,
                                                                   random_state=127))]))],
                   final_estimator=LogisticRegression(verbose=False), n_jobs=-1,
                   verbose=False)

In [23]:
corr_df = pd.DataFrame()

for model, (name, _) in zip(stacking_classifier.estimators_, stacking_classifier.estimators):
    preprocessed = stacking_classifier.estimators[0][1].steps[0][1].fit(X_train, y_train).transform(X_valid)
    print(f'{name:<15} accuracy: ', round(accuracy_score(model.predict(X_valid), y_valid), 6))

    corr_df[name] = model.predict(X_valid)


ExtraTrees      accuracy:  0.843575
XGBoost         accuracy:  0.865922
LightGBM        accuracy:  0.843575
CatBoost        accuracy:  0.854749
Random_forest   accuracy:  0.832402
Tabular_network accuracy:  0.815642


In [24]:
corr_df.corr().style.background_gradient(cmap="RdYlGn")

,ExtraTrees,XGBoost,LightGBM,CatBoost,Random_forest,Tabular_network
ExtraTrees,1.000000,0.827543,0.900528,0.766292,0.950166,0.794027
XGBoost,0.827543,1.000000,0.852224,0.813269,0.827043,0.774673
LightGBM,0.900528,0.852224,1.000000,0.766292,0.900017,0.794027
CatBoost,0.766292,0.813269,0.766292,1.000000,0.743336,0.677590
Random_forest,0.950166,0.827043,0.900017,0.743336,1.000000,0.814187
Tabular_network,0.794027,0.774673,0.794027,0.677590,0.814187,1.000000


In [25]:
print('ensemble score:', round(accuracy_score(stacking_classifier.predict(X_valid), y_valid), 6))

ensemble score: 0.843575


In [26]:
test_df.reset_index(names='PassengerId', inplace=True)

test = test_df[model_columns].drop(columns=['PassengerId'], errors='ignore').copy()

In [27]:
max_num = get_max_num()

In [28]:
submit_prefix = 'sc_'
predict_test = stacking_classifier.predict(test)

# Сохранение предсказаний в файл
submit_csv = f'{submit_prefix}submit_{max_num:03}{LOCAL_FILE}.csv'
file_submit_csv = PREDICTIONS_DIR.joinpath(submit_csv)
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'],
                           'Survived': predict_test.flatten()})
submission.to_csv(file_submit_csv, index=False)

In [29]:
# Random_forest сильно коррелирует с другими моделями, поэтому он снижает качество ансамбля.
# Попробуем его убрать

# список базовых моделей
estimators = [

#     ("ExtraTrees",  make_pipeline(preprocessor, ExtraTreesClassifier(**params_et))),
    
    ("XGBoost", xgb_model),
    
#     ("LightGBM", lgbm_model),
    
    ("CatBoost", cat_model),
    
    ("Random_forest",  make_pipeline(preprocessor, RandomForestClassifier(**params_rf))),
    
#     ("Tabular_network",  make_pipeline(preprocessor, TabularClassifier(**params_nn))),    

]


# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False),
    # final_estimator=RandomForestClassifier(n_estimators = 10_000,
                                           # max_depth = 5,
                                           # verbose=False),
    n_jobs=-1,
    verbose=False,
)

meta_model.fit(X_train, y_train)

print('ensemble score:', round(accuracy_score(meta_model.predict(X_valid), y_valid), 6))

# Без Random_forest:              0.877095
# Без ExtraTrees:                 0.860335
# Без ExtraTrees и Random_forest: 0.860335

ensemble score: 0.871508


In [30]:
submit_prefix = 'mm_'
predict_testm = meta_model.predict(test)

# Сохранение предсказаний в файл
submit_csv = f'{submit_prefix}submit_{max_num:03}{LOCAL_FILE}.csv'
file_submit_csv = PREDICTIONS_DIR.joinpath(submit_csv)
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'],
                           'Survived': predict_testm.flatten()})
submission.to_csv(file_submit_csv, index=False)

# Комментарии

* 📈 Да, скор ансамбля вырос, но есть много **"но"** у этой реализации
* ⚠️ Тут в качестве мета-модели использовалась `LogisticRegression`, что по сути является обычным блендингом, но с кросс-валидацией.
* 🧩 Слабые или похожие модели мешают ансамблю поднять скор (Если убрать `RandomForest` скор поднимется)
* 🍏 Стекинг можно усложнить, подавая мета-модели еще признаки при этом используя более сложную meta-модедь.
* 🤔 Тогда в зависимости от свойств объекта, мета модели, такие как `RandomForestClassifier` могут принимать решение оптимальнее.
* ☹️ В рамках `pipeline` в `Sklearn` это сделать сложнее. Надо взять что-то другое.

* Не все можно запихнуть в `pipeline`. Например `eval_set` для `early-stopping`а или класс `train` от `LightGBM`

# <center> 📚 Дополнительная литература
- [Статья Александра Дьяконова про Стекинг и Блендинг](https://alexanderdyakonov.wordpress.com/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/)
- [Пример решения, где  `LigthGBM` в качестве мета модели](https://www.youtube.com/watch?v=aMlpeDOjib8)